In [ ]:
import os
import sys
import shutil

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from ase import Atoms
from ase.optimize import LBFGS
from HybridFF import HybridFF
from ANA2B import ANA2B
from Utilities import show_results, write_xyz, validate
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM
from scipy.stats import spearmanr

In [ ]:
DATA_ICE13 = np.load('../../../data/test_sets/ICE13/ICE13.npy', allow_pickle=True).item()
DATA_X23 = np.load('../../../data/test_sets/x23_mini.npy', allow_pickle=True).item()

X23_reference = {
    'CYTSIN01': -163.5,
    'UREAXX12': -102.1,
    'HXMTAM10': -84.1,
    'PYRZOL05': -78.8,
    'CYHEXO': -90.0,
    'ACETAC07': -73.6,
    'ADAMAN08': -71.8,
    'nh3': -38.7,
    'ANTCEN09': -110.4,
    'BENZEN01': -54.8,
    'co2': -29.4,
    'CYANAM01': -81.5,
    'ECARBM01': -88.2,
    'FORMAM02': -81.1,
    'IMAZOL04': -90.4,
    'NAPHTA23': -81.3,
    'OXALAC03': -98.8,
    'OXALAC04': -96.8,
    'PYRAZI01': -64.3,
    'TRIZIN': -62.6,
    'TROXAN11': -64.6,
    'SUCACB03': -130.1,
    'URACIL': -136.2,
}

for key in X23_reference:
    DATA_X23[key]['ref'] = X23_reference[key]
DATA = {**DATA_X23, **DATA_ICE13}

In [ ]:
SMEAR = -0.39
S = -int(SMEAR * 100)
REF_DATA = np.load(f'../../../data/test_sets/BENCHMARK_DATA_S{S}2R_D3PBE0.npy', allow_pickle=True).item()

In [ ]:
hybrid_ff = HybridFF()


In [ ]:
ref_key = 'S66x8'
energy_target, energy_predicted = validate(hybrid_ff.ANA, ref_key, REF_DATA)
show_results(energy_target, energy_predicted, ref_key, show_plot=True)

In [ ]:
ref_key = 'S7L_CC'
energy_target, energy_predicted = validate(hybrid_ff.ANA, ref_key, REF_DATA)
show_results(energy_target, energy_predicted, ref_key, show_plot=True)

In [ ]:
RESULTS = {}
MINIMIZED_STRUCTURES = np.load(f'data/MINIMIZED_STRUCTURES_S{S}_ALPHAGNN2R.npy', allow_pickle=True).item()
MONOMER_ENERGIES = np.load('data/MONOMER_ENERGIES.npy', allow_pickle=True).item()

In [ ]:
key = 'co2'
RESULTS[key] = {}
print(key, DATA[key]['ref'])
elements_uc = DATA[key]['uc_symbols']
coords_uc = DATA[key]['uc_coordinates'][None]
lattice = DATA[key]['cell'][None]
mol_size = len(DATA[key]['monomer_symbols'])
hybrid_ff.initialize(elements_uc, coords_uc, lattice, mol_size)
output = hybrid_ff.get_components(coords_uc, elements_uc, lattice)
print(f'Total: {output["V_total"][0].numpy()}')
print(f'Intermolecular: {output["V_inter"][0].numpy()}')

RESULTS[key]['ref'] = DATA[key]['ref']
RESULTS[key]['lat'] = output["V_inter"][0].numpy()

In [ ]:
for key in X23_reference:    
    if key not in ['co2']:
        RESULTS[key] = {}
        print(key, DATA[key]['ref'])
        elements_uc = MINIMIZED_STRUCTURES[key]['elements_uc']
        mol_size = len(DATA_X23[key]['monomer_symbols'])
        coords_uc = MINIMIZED_STRUCTURES[key]['coords_uc']
        lattice = MINIMIZED_STRUCTURES[key]['lattice']
        atoms = hybrid_ff.initialize(elements_uc, coords_uc, lattice, mol_size)
        optimizer = LBFGS(atoms) 
        optimizer.run(fmax=1 / EV_TO_KJ)       
        coords_uc = atoms.get_positions()[None].astype(np.float32)
        V_total, dV_total = hybrid_ff(coords_uc, lattice, grad=True)
        E_lat = V_total[0] /  hybrid_ff._n_molecules_uc - MONOMER_ENERGIES[key]       
        print(f'Lattice Energy [kJ/mol]: {E_lat}')
        print(f'Max Gradient: {tf.reduce_max(dV_total).numpy()}')
        
        RESULTS[key]['ref'] = DATA[key]['ref']
        RESULTS[key]['lat'] = E_lat
        RESULTS[key]['structure'] = coords_uc
        RESULTS[key]['elements_uc'] = elements_uc
        RESULTS[key]['lattice'] = lattice
        MINIMIZED_STRUCTURES[key]['coords_uc'] = coords_uc

In [ ]:
for key in DATA_ICE13:
    print(key, DATA[key]['ref'])
    RESULTS[key] = {}
    elements_uc = MINIMIZED_STRUCTURES[key]['elements_uc']
    mol_size = len(DATA[key]['monomer_symbols'])
    coords_uc = MINIMIZED_STRUCTURES[key]['coords_uc']
    lattice = MINIMIZED_STRUCTURES[key]['lattice']
    atoms = hybrid_ff.initialize(elements_uc, coords_uc, lattice, mol_size) 
    optimizer = LBFGS(atoms)
    optimizer.run(fmax=1 / EV_TO_KJ)
    coords_uc = atoms.get_positions()[None].astype(np.float32)
    V_total, dV_total = hybrid_ff(coords_uc, lattice, grad=True)
    E_lat = V_total[0] /  hybrid_ff._n_molecules_uc - MONOMER_ENERGIES['ICE']
    print(f'Lattice Energy: {E_lat}')
    print(f'Max Gradient: {tf.reduce_max(dV_total).numpy()}')    
    RESULTS[key]['ref'] = DATA[key]['ref']
    RESULTS[key]['lat'] = E_lat
    RESULTS[key]['structure'] = coords_uc
    RESULTS[key]['elements_uc'] = elements_uc
    RESULTS[key]['lattice'] = lattice
    MINIMIZED_STRUCTURES[key]['coords_uc'] = coords_uc

In [ ]:
np.save('RESULTS_IND', RESULTS)

In [ ]:
references, predictions, names = [], [], []
for key in X23_reference:
    references.append(RESULTS[key]['ref'])
    predictions.append(RESULTS[key]['lat'])
    names.append(key)
references, predictions = np.array(references), np.array(predictions)

In [ ]:
print(spearmanr(references, predictions).statistic)
_, _, _, fig = show_results(references, predictions, '',show_chem_acc=True, show_plot=True, show_mae=False)
plt.show()
_, _, _, fig = show_results(references, predictions, '',show_chem_acc=True, names=names, show_plot=True, show_mae=False)
plt.show()
_, _, _, fig = show_results(references, predictions, 'X23', names=names, show_chem_acc=True, show_plot=True)


In [ ]:
references, predictions, names = [], [], []
for key in DATA_ICE13:
    references.append(RESULTS[key]['ref'])
    predictions.append(RESULTS[key]['lat'])
    names.append(key)
references, predictions = np.array(references), np.array(predictions)

In [ ]:
print(spearmanr(references, predictions).statistic)
show_results(references, predictions, '',show_chem_acc=True, show_plot=True, show_mae=False)
plt.show()
show_results(references, predictions, '',show_chem_acc=True, names=names, show_plot=True, show_mae=False)
plt.show()
show_results(references, predictions, 'ICE13', names=names, show_chem_acc=True, show_plot=True)


In [ ]:
RESULTS = np.load('RESULTS_IND.npy', allow_pickle=True).item()

In [ ]:
tab_string = ''
N_DIGITS = 1
for key in X23_reference:
    ref = np.round(RESULTS[key]['ref'], N_DIGITS)
    pred = np.round(np.array(RESULTS[key]['lat']), N_DIGITS)
    delta = pred - ref
    tab_string += f'{key.upper()} & {ref:{4}.{N_DIGITS}f} & {pred:{4}.{N_DIGITS}f} & {delta:{4}.{N_DIGITS}f}\\\\\n'
print(tab_string)

In [ ]:
tab_string = ''
N_DIGITS = 1
keys = sorted(DATA_ICE13.keys())
for key in keys:
    ref = np.round(RESULTS[key]['ref'], N_DIGITS)
    pred = np.round(np.array(RESULTS[key]['lat']), N_DIGITS)
    delta = pred - ref
    tab_string += f'{key} & {ref:{4}.{N_DIGITS}f} & {pred:{4}.{N_DIGITS}f} & {delta:{4}.{N_DIGITS}f}\\\\\n'
print(tab_string)